# Run on the colab GPU!

# Prepare some staff

In [121]:
!pip install datasets transformers accelerate

In [122]:
import random
import string

random_state = 100
random.seed(random_state)

In [123]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Will use device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print(f"We will use device: CPU")

Will use device: Tesla T4


In [124]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [125]:
print_gpu_utilization()

GPU memory occupied: 7576 MB.


In [126]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [127]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "sberbank-ai/ruRoberta-large"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [128]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNor

In [129]:
print_gpu_utilization()

GPU memory occupied: 7576 MB.


In [130]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

PreTrainedTokenizerFast(name_or_path='sberbank-ai/ruRoberta-large', vocab_size=50257, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [131]:
print_gpu_utilization()

GPU memory occupied: 7576 MB.


In [132]:
text = f"ультразвуковой исследование {tokenizer.mask_token} полость"
text

'ультразвуковой исследование <mask> полость'

In [133]:
tokenizer.decode(tokenizer(text)["input_ids"])

'<s>ультразвуковой исследование<mask> полость</s>'

In [134]:
# Tokenizer should return [1, index, 2], but it didn't
tokenizer("хронический")["input_ids"]

[1, 291, 712, 20059, 2]

In [135]:
#Let's train our tokenizer to correct tokenize medical words
# Source https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb

We can't use all known words from dictionary, because then the embedding in model become too large and don't fit into the GPU memory (4 GB for NVIDIA GeForce GTX 1650 or 24 GB for Tesla K80 on Colab). So let's use all words from existed anamnesis, and each word should appear at least two times.

In [136]:
import pandas as pd

In [137]:
path_to_processed_anamnesis = "/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/almazov_isa_anamnesis.csv"
processed_anamnesis = pd.read_csv(path_to_processed_anamnesis, header=None, names=["anamnes"])
print(len(processed_anamnesis))
processed_anamnesis.head()

2516


,anamnes
0,хронический заболевание анамнез сахарный диабе...
1,псориатический артрит холецистэктомия наследст...
2,молодость работать трюм ледокол работа краска ...
3,хронический заболевание анамнез остеохондроз п...
4,перенести состояние травма перелом правый плеч...


In [138]:
processed_anamnesis["anamnes"] = processed_anamnesis["anamnes"].map(lambda x: x.replace('ё', 'е'))
processed_anamnesis.head()

,anamnes
0,хронический заболевание анамнез сахарный диабе...
1,псориатический артрит холецистэктомия наследст...
2,молодость работать трюм ледокол работа краска ...
3,хронический заболевание анамнез остеохондроз п...
4,перенести состояние травма перелом правый плеч...


In [139]:
processed_anamnesis = processed_anamnesis.anamnes.values

In [140]:
import numpy as np
anamnesis_words = np.concatenate(list(map(lambda x: x.split(), processed_anamnesis)))
len(anamnesis_words)

199484

In [141]:
from re import search
from collections import Counter

MIN_COUNT_WORD = 0
unique_anamnesis_words = [ val for val, i in Counter(anamnesis_words).items() if i > MIN_COUNT_WORD and len(val) > 2 and (not search("[^а-яА-Я]", val))]
len(anamnesis_words)

199484

In [142]:
anamnesis_words = [word for word in anamnesis_words if word in unique_anamnesis_words]

In [143]:
len(anamnesis_words)

189009

In [144]:
processed_words = anamnesis_words 

In [145]:
batch_size = 32
def batch_iterator(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i : i + batch_size]

In [146]:
tokenizer.vocab_size

50257

In [147]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(processed_words, batch_size), vocab_size=len(unique_anamnesis_words))

In [148]:
# path_to_processed_words_dict = "/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/processed_lemmatized_all_dict.txt"
# words_dict = pd.read_csv(path_to_processed_words_dict, index_col=0)
# words_dict.head()

In [149]:
# new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(words_dict.word, batch_size), vocab_size=len(words_dict))

In [150]:
new_tokenizer.vocab_size

7592

In [151]:
# Tokenizer should return [1, index, 2], and we got it
new_tokenizer("хронический")["input_ids"]

[1, 150, 2]

In [152]:
new_tokenizer("пациент отрицать хронический рак")["input_ids"]

[1, 2196, 192, 150, 802, 2]

In [153]:
new_tokenizer("пациентотрицатьхроническийрак")["input_ids"]

[1, 2196, 192, 150, 802, 2]

In [154]:
# save tokenizer
path_to_tokenizer = "/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer" #"../../data/ml/ruBert-large-fine-tuned"
new_tokenizer.save_pretrained(path_to_tokenizer)

('/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer/tokenizer.json')

In [155]:
tokenizer = AutoTokenizer.from_pretrained(path_to_tokenizer)
tokenizer

PreTrainedTokenizerFast(name_or_path='/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-tokenizer', vocab_size=7592, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [156]:
tokenizer(text)["input_ids"]

[1, 2225, 666, 4, 495, 2]

In [157]:
tokenizer.decode(tokenizer(text)["input_ids"])

'<s>ультразвуковойисследование<mask>полость</s>'

In [158]:
model.resize_token_embeddings(len(tokenizer))

Embedding(7592, 1024)

In [159]:
import torch

inputs = tokenizer(text, return_tensors="pt")
print(inputs["input_ids"])
print(inputs)
token_logits = model(**inputs).logits# Find the location of <mask> and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
mask_token_logits = torch.softmax(mask_token_logits, dim=1)
# Pick the <mask> candidates with the highest logits
top_20 = torch.topk(mask_token_logits, 20, dim=1)
top_20_tokens = zip(top_20.indices[0].tolist(), top_20.values[0].tolist())

for token, score  in top_20_tokens:
    print(f"{text.replace(tokenizer.mask_token, tokenizer.decode([token]))}, score: {score}")

tensor([[   1, 2225,  666,    4,  495,    2]])
{'input_ids': tensor([[   1, 2225,  666,    4,  495,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
ультразвуковой исследование инфаркт полость, score: 0.09932681173086166
ультразвуковой исследование уплот полость, score: 0.083730548620224
ультразвуковой исследование � полость, score: 0.05445001646876335
ультразвуковой исследование рование полость, score: 0.04204767197370529
ультразвуковой исследование операция полость, score: 0.03416870906949043
ультразвуковой исследование уровень полость, score: 0.02747410722076893
ультразвуковой исследование � полость, score: 0.026905914768576622
ультразвуковой исследование тр полость, score: 0.026378411799669266
ультразвуковой исследование сред полость, score: 0.01924346573650837
ультразвуковой исследование циал полость, score: 0.017405129969120026
ультразвуковой исследование жив полость, score: 0.017222872003912926
ультразвуковой исследование фи полость, score: 0.017201796174049377
ультразву

In [160]:
import numpy as np

np.random.shuffle(processed_anamnesis)
train = processed_anamnesis[:int((len(processed_anamnesis)+1)*.8)]
test = processed_anamnesis[int((len(processed_anamnesis)+1)*.80):]
len(train)

2013

In [161]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_dict({"text": train})
test_dataset = Dataset.from_dict({"text": test})

In [162]:
anamnesis_dataset = DatasetDict({"train": train_dataset, "test": test_dataset})
anamnesis_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2013
    })
    test: Dataset({
        features: ['text'],
        num_rows: 503
    })
})

In [163]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    #if tokenizer.is_fast:
    #    result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_datasets = anamnesis_dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2013
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 503
    })
})

In [164]:
chunk_size = 500

In [165]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples['input_ids'])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [166]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets.set_format("pt")
lm_datasets

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 347
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 80
    })
})

In [167]:
tokenizer.decode(lm_datasets["train"][100]["input_ids"])

'пивонескольконеделяводкамесяцобразжизньслужитьпогранвойскоработатьводительэлектромеханикребенок</s><s>перенестисостояниеоперациягповодкатарактареспираторныйзаболеваниехроническийзаболеваниеанамнезстенокардиянапряжениесахарныйдиабеттипсубкомпенсироватьгинекологическийанамнезродыменопаузанаследственностьотяготитьстраховойанамнезработатьинвалидностьаллергологическийанамнезособенностьвредныйпривычкаотрицатьусловиетрудработатьштукатурпрофессиональныйвредностьотрицатьобразжизньбытовойусловиеудовлетворительныйпитаниерегулярный</s><s>хроническийзаболеваниеанамнезострыйнижнийэлевация</s><s>растиразвиватьсядетствоотставатьсверстникмалярияболетьвенерическийзаболеваниеотрицатьгепатитгодаллергологическийанамнезаллергическийреакцияотмечатьнаследственностьотяготитьвредныйпривычкакуритьгодпачкасигаретаденьалкогользлоупотреблятьстраховойанамнезработатьблнужныйусловиежизньудовлетворительныйпрофессиональныйвредностьзаболеваниехроническийгастритремиссияаппендэктомиядетствооблитерирующийатросклерозсосудни

In [168]:
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask

data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm_probability=0.15)

#### Without spaces but it's ok (I hope)

In [169]:
from transformers import TrainingArguments

# Let's fine-tune the model
batch_size = 64

training_args = TrainingArguments(
    output_dir=f"{model_checkpoint}-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=25,
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=batch_size,
    gradient_checkpointing=True,
    per_device_eval_batch_size=1,
    #no_cuda=True,
    fp16=True
)

In [170]:
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader
from torch.optim import AdamW

train_dataloader = DataLoader(lm_datasets["train"],
                        batch_size=training_args.per_device_train_batch_size,
                        collate_fn=data_collator)

test_dataloader = DataLoader(lm_datasets["test"],
                        batch_size=training_args.per_device_train_batch_size,
                        collate_fn=data_collator)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator(fp16=training_args.fp16)
adam_w_optim = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
model, optimizer, train_dataloader, test_dataloader = accelerator.prepare(model, adam_w_optim, train_dataloader, test_dataloader)


In [171]:
tokenizer.decode(lm_datasets["train"][0]["input_ids"])

'<s>женатыйребенокработатьармияслужитьперенестисостояниеоперацияаппендэктомиягодхроническийзаболеваниеанамнезварикозныйболезньвенонижнийконечностьнаследственностьотяготитьстраховойанамнезработатьбольничныйлистнужныйаллергологическийанамнезособенностьвредныйпривычкакурениегодкуритьнесколькосигаретаденьобразжизньбытовойусловиеудовлетворительныйгиподинамия</s><s>перенестисостояниеоперацияповодножевойранениеживотаппендэктомиягодхроническийзаболеваниеанамнезнаследственностьзаболеваниеродственникматьстраховойанамнезработатьбольничныйлистинвалидностьаллергологическийанамнезособенностьвредныйпривычкакурениепачкаденьалкогольпраздникусловиетрудработатьводительобразжизньбытовойусловиеудовлетворительныйпитаниерегулярный</s><s>ознакомитьдополнение</s><s>страдатьхроническийобструктивныйбронхитзаболеваниепеченьпочкаязвенныйболезнькровотечениеанамнезотрицатьоперироватьповодаденомапредстательныйжелезастраховойанамнезработатьбольничныйлисталлергологическийанамнезособенностьвредныйпривычкакурениепачкаден

In [172]:
for item in train_dataloader:
  print(tokenizer.decode(item["input_ids"][0]))
  break


<s>женатыйребенокработатьармияслужитьперенестисостояниеоперацияаппендэктомиягодхроническийзаболеваниеанамнез<mask>болезнь<mask>нижнийконечностьнаследственностьотяготитьстраховойанамнезработатьбольничныйлистнужныйаллергологическийанамнезособенностьвредныйпривычкакурениегодкуритьнесколько<mask>день<mask>жизньбытовойусловиеудовлетворительныйгиподинамия</s><s>перенестисостояниеоперацияповодножевойранениеживотаппендэктомиягодхроническийзаболеваниеанамнезнаследственностьзаболеваниеродственникматьстраховойанамнезработатьбольничныйлист<mask>аллергологическийанамнез<mask>вредныйпривычкакурениепачкаденьалкогольпраздникусловие<mask>работатьводительобразжизньбытовойусловие<mask>питание<mask></s><s>ознакомитьдополнение</s><s><mask>хроническийобструктивныйбронхитзаболеваниепеченьпочка<mask>болезнькровотечение<mask>отрицатьоперироватьповодаденомапредстательныйжелезастраховойанамнез<mask>больничныйлисталлергологическийанамнез<mask>вредныйпривычкакурениепачкадень</s><s>хроническийзаболеваниеанамнезязве

/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  "DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. "


In [173]:
print_gpu_utilization()

GPU memory occupied: 7576 MB.


In [174]:
import math
from tqdm.auto import tqdm


for epoch in range(training_args.num_train_epochs):
    progress_bar = tqdm(range(len(train_dataloader)))

    model.train()
    for step, batch in enumerate(train_dataloader, start=1):
        loss = model(**batch).loss
        loss = loss / training_args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step % training_args.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        progress_bar.update(1)

    print_gpu_utilization()
    print(f"EVAL EPOCH {epoch}")
    progress_bar = tqdm(range(len(test_dataloader)))
    model.eval()
    losses = []
    for step, batch in enumerate(test_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))
        progress_bar.update(1)

    losses = torch.cat(losses)
    losses = losses[: len(test_dataloader)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")


  0%|          | 0/347 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  "DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. "


GPU memory occupied: 7576 MB.
EVAL EPOCH 0


  0%|          | 0/80 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 1024.6310732584982


...

>>> Epoch 23: Perplexity: 37.64441325676468


  0%|          | 0/347 [00:00<?, ?it/s]

GPU memory occupied: 7578 MB.
EVAL EPOCH 24


  0%|          | 0/80 [00:00<?, ?it/s]

>>> Epoch 24: Perplexity: 46.16719265578688


In [175]:
import torch

inputs = tokenizer(text, return_tensors="pt")
print(inputs["input_ids"])
print(inputs)
token_logits = model(inputs["input_ids"].cuda(), inputs["attention_mask"].cuda()).logits
# Find the location of <mask> and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
mask_token_logits = torch.softmax(mask_token_logits, dim=1)
# Pick the <mask> candidates with the highest logits
top_20 = torch.topk(mask_token_logits, 20, dim=1)
top_20_tokens = zip(top_20.indices[0].tolist(), top_20.values[0].tolist())

for token, score  in top_20_tokens:
    print(f"{text.replace(tokenizer.mask_token, tokenizer.decode([token]))}, score: {score}")

tensor([[   1, 2225,  666,    4,  495,    2]])
{'input_ids': tensor([[   1, 2225,  666,    4,  495,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
ультразвуковой исследование год полость, score: 0.034981559962034225
ультразвуковой исследование работать полость, score: 0.030511463060975075
ультразвуковой исследование течение полость, score: 0.026612568646669388
ультразвуковой исследование день полость, score: 0.021635863929986954
ультразвуковой исследование мл полость, score: 0.014300432987511158
ультразвуковой исследование госпитализация полость, score: 0.009526141919195652
ультразвуковой исследование средний полость, score: 0.008572600781917572
ультразвуковой исследование динамика полость, score: 0.007654470857232809
ультразвуковой исследование состояние полость, score: 0.007624628953635693
ультразвуковой исследование образование полость, score: 0.007506419904530048
ультразвуковой исследование правый полость, score: 0.007190709467977285
ультразвуковой исследование левый поло

In [176]:
sought_after_token = "брюшной"
sought_after_token_id = tokenizer.encode(sought_after_token, add_special_tokens=False)[0] 
sought_after_token_id


965

In [177]:
token_score = mask_token_logits[:, sought_after_token_id]
print(f"Score of {sought_after_token}: {mask_token_logits[:, sought_after_token_id]}")

Score of брюшной: tensor([0.0011], device='cuda:0', grad_fn=<SelectBackward0>)


In [178]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model=model.cpu(), tokenizer=tokenizer)

In [179]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> ультразвуковойисследованиегодполость
>>> ультразвуковойисследованиеработатьполость
>>> ультразвуковойисследованиетечениеполость
>>> ультразвуковойисследованиеденьполость
>>> ультразвуковойисследованиемлполость


In [180]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/MedSpellChecker/ruBert-large-fine-tuned-model")